In [1]:
# Загрузка всего необходимого
!pip install torch
!pip install fitz
!pip install PyMuPDF
!pip install transformers
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datetime import datetime
import json
import torch
import fitz
import re
model = T5ForConditionalGeneration.from_pretrained('IlyaGusev/rut5_base_sum_gazeta')
tokenizer = T5Tokenizer.from_pretrained('IlyaGusev/rut5_base_sum_gazeta')
device = "cuda" if torch.cuda.is_available() else "cpu"

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\Vadim\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
#Функция загрузки резюме
def get_cv(CV):
    CV_title = CV
    #Открываем PDF-файл
    with fitz.open(CV_title) as doc:
        extracted_text = ''
        # Проходим по каждой странице
        for page in doc:
            # Извлекаем только текст, игнорируя изображения
            extracted_text += page.get_text("text")
    return extracted_text
            
# Функция для суммаризации одного блока текста
def summarize_text(text):
    input_ids = tokenizer(
    [text],
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    max_length=400,
    return_tensors="pt")["input_ids"].to(device)

    output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=3,
    num_beams=10,
    length_penalty=2,
    early_stopping=True)[0]
    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

#Разбиение текста на блоки
def split_text(text, max_length=300):
    sentences = text.split(". ")
    current_chunk = []
    current_length = 0
    chunks = []
    for sentence in sentences:
        length = len(tokenizer.encode(sentence))
        if current_length + length <= max_length:
            current_chunk.append(sentence)
            current_length += length
        else:
            chunks.append(". ".join(current_chunk) + ".")
            current_chunk = [sentence]
            current_length = length

    if current_chunk:
        chunks.append(". ".join(current_chunk) + ".")
    
    if((len(tokenizer.encode(chunks[len(chunks)-1])))<100):
        chunks[len(chunks)-2] += " "
        chunks[len(chunks)-2] += chunks[len(chunks)-1]
        chunks.remove(chunks[len(chunks)-1])
    return chunks


#Функция запуска суммаризации
def summarize_long_text(long_text):
    chunks = split_text(long_text)
    summarized_chunks = [summarize_text(chunk) for chunk in chunks]
    final_summary = "\n".join(summarized_chunks)
    return final_summary

#Основаня функция
def summarize(CV):
    extracted_cv = get_cv(CV)
    short_summary = summarize_long_text(extracted_cv)
    final_summary = summarize_text(short_summary)
    print(CV[:len(CV)-4])
    print("Суммаризация резюме:")
    print(final_summary)
    

In [4]:
summarize("Фомин Марк Кириллович.pdf")

Фомин Марк Кириллович
Суммаризация резюме:
Занятость: Полная Желаемая зарплата: от 45 000 руб. Дата рождения: 11.12.1989 г. (32 года) О себе Личные качества Умение быстро ориентироваться в ситуации, требовательность к себе и окружающим, стремление к организации четкого и сбалансированного производственного процесса.


In [5]:
summarize("Осипов Артём Михайлович.pdf")

Осипов Артём Михайлович
Суммаризация резюме:
Обязанности: Составление штатного расписания, нормирование оплаты труда, ведение бухгалтерского и налогового учета в программе 1С:7. Опыт работы в структуре системных предприятий холдинга: Работа с первичной документацией, сверка взаиморасчетов с контрагентами и размещение на Госзакупках.


In [6]:
summarize("Соколова Олеся Артуровна.pdf")

Соколова Олеся Артуровна
Суммаризация резюме:
Институт лингвистики и международных коммуникаций в Челябинске окончила курс переводчика в сфере профессиональной коммуникации. О себе Личные качества Настойчивость, проявление инициативы, активность, обучаемость, исполнительность.
